<a href="https://colab.research.google.com/github/Shriansh16/Generative-AI/blob/main/pinecone_db.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain
!pip install pypdf
!pip install pinecone-client
!pip install openai
!pip install tiktoken  #for embedding

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.5/308.5 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.8/122.8 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.5/215.5 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [2]:
pip install -U langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 5.6 MB/s eta 0:00:00


In [3]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.vectorstores import Pinecone
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
import os

In [4]:
!mkdir pdfs

In [5]:
loader=PyPDFDirectoryLoader('pdfs')

In [6]:
data=loader.load()

In [7]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)

In [8]:
text_chunks = text_splitter.split_documents(data)

In [9]:
len(text_chunks)

91

In [10]:
print(text_chunks[0].page_content)

Provided proper attribution is provided, Google hereby grants permission to
reproduce the tables and figures in this paper solely for use in journalistic or
scholarly works.
Attention Is All You Need
Ashish Vaswani∗
Google Brain
avaswani@google.comNoam Shazeer∗
Google Brain
noam@google.comNiki Parmar∗
Google Research
nikip@google.comJakob Uszkoreit∗
Google Research
usz@google.com
Llion Jones∗
Google Research
llion@google.comAidan N. Gomez∗ †
University of Toronto


In [11]:
print(text_chunks[0])

page_content='Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.comNoam Shazeer∗\nGoogle Brain\nnoam@google.comNiki Parmar∗\nGoogle Research\nnikip@google.comJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.comAidan N. Gomez∗ †\nUniversity of Toronto' metadata={'source': 'pdfs/transformer_architecture.pdf', 'page': 0}


In [12]:
import os
os.environ["OPENAI_API_KEY"]=""

In [13]:
embedding=OpenAIEmbeddings()

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [14]:
len(embedding.embed_query("How are you"))

1536

In [15]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key='')


In [18]:
index_name = "docs-quickstart-index"

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=1536,
        metric="cosine",
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        )
    )


In [21]:
vectors= [embedding.embed_query(chunk.page_content) for chunk in text_chunks]

In [22]:
vector_data = [
    {"id": f"vec{i+1}", "values": embedding}
    for i, embedding in enumerate(vectors)
]

In [23]:
index = pc.Index(index_name)


index.upsert(
    vectors=vector_data,
    namespace="ns1"
)

{'upserted_count': 91}

In [24]:
query="who is the founder of transformer?"
query_vector=embedding.embed_query(query)

In [28]:
query_results1 = index.query(
    namespace="ns1",
    vector=query_vector,
    top_k=3,
    include_values=True
)

In [29]:
query_results1

{'matches': [{'id': 'vec5',
              'score': 0.798092484,
              'values': [-0.0182210989,
                         -0.00850455649,
                         0.0270217657,
                         -0.0206037518,
                         0.0225181393,
                         0.0332194194,
                         -0.0187306833,
                         0.00620453898,
                         0.00156921125,
                         -0.0339355916,
                         0.0368002839,
                         0.00255481014,
                         0.0124848271,
                         0.0342661329,
                         -0.00969588384,
                         -0.0302996356,
                         0.0108321197,
                         0.0306852683,
                         -0.0118857212,
                         -0.000547889736,
                         -0.0318972543,
                         -0.00105360104,
                         -0.00731323054,
                  

In [32]:
def answer_query(query):
    query_vector = embedding.embed_query(query)
    response = index.query(
        vector=query_vector,
        top_k=5,  # Number of relevant vectors to retrieve
        namespace="ns1"
    )
    relevant_ids = [match['id'] for match in response['matches']]
    relevant_chunks = [text_chunks[int(id[3:]) - 1] for id in relevant_ids]  # Adjust according to your ID format
    answer = " ".join(chunk.page_content for chunk in relevant_chunks)
    return answer

In [34]:
query = "who is the founder of transformer?"
answer = answer_query(query)
print(answer)

the effort to evaluate this idea. Ashish, with Illia, designed and implemented the first Transformer models and
has been crucially involved in every aspect of this work. Noam proposed scaled dot-product attention, multi-head
attention and the parameter-free position representation and became the other person involved in nearly every
detail. Niki designed, implemented, tuned and evaluated countless model variants in our original codebase and aidan@cs.toronto.eduŁukasz Kaiser∗
Google Brain
lukaszkaiser@google.com
Illia Polosukhin∗ ‡
illia.polosukhin@gmail.com
Abstract
The dominant sequence transduction models are based on complex recurrent or
convolutional neural networks that include an encoder and a decoder. The best
performing models also connect the encoder and decoder through an attention
mechanism. We propose a new simple network architecture, the Transformer, 7 Conclusion
In this work, we presented the Transformer, the first sequence transduction model based entirely on
attention,

In [35]:
llm = OpenAI()

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [48]:
from langchain.vectorstores import Pinecone

text_field = "text"



vectorstore = Pinecone(
    index, embedding.embed_query, text_field
)

/usr/local/lib/python3.10/dist-packages/langchain_community/vectorstores/pinecone.py:68: UserWarning: Passing in `embedding` as a Callable is deprecated. Please pass in an Embeddings object instead.
  warnings.warn(


In [54]:
vectorstore.as_retriever()

VectorStoreRetriever(tags=['Pinecone'], vectorstore=<langchain_community.vectorstores.pinecone.Pinecone object at 0x7d7426fc77c0>)

In [38]:
index = pc.Index(index_name)

In [39]:
index

In [60]:
query = "what is at chun k 2"

vectorstore.similarity_search(
    query,  # our search query
    k=3  # return 3 most relevant docs
)

[]

In [58]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

# completion llm
llm = ChatOpenAI(

    model_name='gpt-3.5-turbo',
    temperature=0.0
)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever()
)

In [61]:
qa.run(query)

'I\'m sorry, but I don\'t have enough context to determine what "chun k 2" refers to. Could you please provide more information or clarify your question?'

In [55]:
print(index.describe_index_stats())

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 91}},
 'total_vector_count': 91}
